## ANÁLISE DO CLUSTER 2 DA UP GLÓRIA

In [1]:
# Bibliotecas
import numpy as np
import pandas as pd

In [2]:
# Upload do arquivo para análise
gloria_3_caminho = R"C:\Users\Administrador\Downloads\TCC-20240110T120732Z-001\Dados_UP Gloria\Cluster 3\Cluster3_GLORIA_ALTERACOES.xlsx"
gloria_3 = pd.read_excel(gloria_3_caminho)
gloria_3

,Linha,Sublinha,PC,SIU,id_hex,mes,ano,Alteracao_QH,Alteracao_IT,pass,Houve aumento passageiro,Tarifa_Aumento,Tarifa_reducao,Cluster
0,4501,1,1,40994,89a88cda3a3ffff,AGO,2022,1,0,28,0,0,0,3
1,4501,1,1,40997,89a88cda3a3ffff,AGO,2022,1,0,41,0,0,0,3
2,4501,1,2,40996,89a88cda3a3ffff,AGO,2022,1,0,2,0,0,0,3
3,4501,1,2,40993,89a88cda3a3ffff,AGO,2022,1,0,5,0,0,0,3
4,4501,1,2,41123,89a88cda337ffff,AGO,2022,1,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,4802A,5,2,41334,89a88cdae17ffff,AGO,2023,0,0,3,0,0,0,3
416,4802A,5,2,40465,89a88cdae8fffff,AGO,2023,0,0,0,0,0,0,3
417,4802A,12,1,41342,89a88cdae17ffff,AGO,2023,0,0,0,0,0,0,3
418,4802A,12,1,41334,89a88cdae17ffff,AGO,2023,0,0,0,0,0,0,3


# Estatísticas descritivas

In [3]:
estatisticas_descritivas = ['Alteracao_QH', 'Alteracao_IT', 'Houve aumento passageiro', 'Tarifa_Aumento', 'Tarifa_reducao']
for col in estatisticas_descritivas:
    gloria_3[col] = gloria_3[col].astype(int)

estatisticas = {}
for col in gloria_3.columns:
   estatisticas[col] = {
        'Total Observações': gloria_3[col].count(),
        '0 - Não': (gloria_3[col] == 0).sum(),
        '1 - Sim': (gloria_3[col] == 1).sum()
    }

estatisticas = pd.DataFrame(estatisticas)
estatisticas


,Linha,Sublinha,PC,SIU,id_hex,mes,ano,Alteracao_QH,Alteracao_IT,pass,Houve aumento passageiro,Tarifa_Aumento,Tarifa_reducao,Cluster
Total Observações,420,420,420,420,420,420,420,420,420,420,420,420,420,420
0 - Não,0,0,0,0,0,0,0,272,396,187,321,385,385,0
1 - Sim,0,192,204,0,0,0,0,148,24,50,99,35,35,0


# Teste V de Cramer
Objetivo: Testar asssociação entre as variáveis

In [4]:
# Pacotes
from scipy.stats import chi2_contingency

In [5]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

In [6]:
columns_to_test = ['Alteracao_QH', 'Alteracao_IT', 'Houve aumento passageiro', 'Tarifa_Aumento', 'Tarifa_reducao']
# Preparando um DataFrame para armazenar os resultados do V de Cramer
cramers_results_selected = pd.DataFrame(index=columns_to_test, columns=columns_to_test)

# Calculando o V de Cramer para cada par de variáveis selecionadas
for col1 in columns_to_test:
    for col2 in columns_to_test:
        cramers_results_selected.loc[col1, col2] = cramers_v(gloria_3[col1], gloria_3[col2])

cramers_results_selected

,Alteracao_QH,Alteracao_IT,Houve aumento passageiro,Tarifa_Aumento,Tarifa_reducao
Alteracao_QH,0.994771,0.096751,0.0,0.207972,0.21415
Alteracao_IT,0.096751,0.977851,0.0,0.026725,0.026725
Houve aumento passageiro,0.0,0.0,0.993376,0.083212,0.117207
Tarifa_Aumento,0.207972,0.026725,0.083212,0.984378,0.057402
Tarifa_reducao,0.21415,0.026725,0.117207,0.057402,0.984378


Análise: Associação fraca a moderada entre alterações no Quadro de Horário e redução da tarifa. Associação entre redução da tarifa e aumento de passageiros

# ANÁLISE BINOMIAL NEGATIVA

In [8]:
gloria3_cam = r"C:\Users\Administrador\Downloads\TCC-20240110T120732Z-001\Dados_UP Gloria\Cluster 3\RendaFreq.xlsx"
gloria3 = pd.read_excel(gloria3_cam)
gloria3

,Hexagono,Bairro,UP,Mes,Aumentou,Quantidade_passageiros(Y),Numero_linhas,Frequencia_HEX,Populacao,Renda_DOMICILIAR,Quantidade_escolas,Cluster
0,89a88cda337ffff,Califórnia,Glória,"agosto, 2022",0,0,1,50,322,503.6,0,3
1,89a88cda3a3ffff,Califórnia,Glória,"agosto, 2022",0,76,1,98,419,588.7,0,3
2,89a88cdae17ffff,Pindorama,Glória,"agosto, 2022",0,95,1,93,461,469.0,0,3
3,89a88cdae8fffff,Pindorama,Glória,"agosto, 2022",0,29,1,93,155,293.4,0,3
4,89a88cdae9bffff,Pindorama,Glória,"agosto, 2022",0,184,1,72,26,257.5,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
61,89a88cda3a3ffff,Califórnia,Glória,"agosto, 2023",1,56,1,97,419,588.7,0,3
62,89a88cdae17ffff,Pindorama,Glória,"agosto, 2023",1,100,1,97,461,469.0,0,3
63,89a88cdae8fffff,Pindorama,Glória,"agosto, 2023",0,25,1,96,155,293.4,0,3
64,89a88cdae9bffff,Pindorama,Glória,"agosto, 2023",0,62,1,57,26,257.5,0,3


In [11]:
#pacotes
import statsmodels.api as sm
from statsmodels.formula.api import negativebinomial

In [12]:
modelo_nb = negativebinomial('Q("Quantidade_passageiros(Y)") ~ Frequencia_HEX + Numero_linhas + Populacao + Renda_DOMICILIAR', data=gloria2)
resultado_nb = modelo_nb.fit()

# Visualizando os resultados
print(resultado_nb.summary())

         Current function value: 5.481142
         Iterations: 23
         Function evaluations: 103
         Gradient evaluations: 91
                           NegativeBinomial Regression Results                            
Dep. Variable:     Q("Quantidade_passageiros(Y)")   No. Observations:                  275
Model:                           NegativeBinomial   Df Residuals:                      270
Method:                                       MLE   Df Model:                            4
Date:                            Sun, 21 Jan 2024   Pseudo R-squ.:                 0.05628
Time:                                    15:46:23   Log-Likelihood:                -1507.3
converged:                                  False   LL-Null:                       -1597.2
Covariance Type:                        nonrobust   LLR p-value:                 8.361e-38
                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------

c:\Users\Administrador\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_optimize.py:1360: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)
c:\Users\Administrador\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Concluí-se que a  frequência tem relação com a quantidade de passageiros, a renda indica pouca influencia. 